In [2]:
# importing dependencies
import pandas as pd
import numpy as np
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from keras.preprocessing import text, sequence
from keras import layers, models, optimizers
import keras
import re
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.metrics import roc_auc_score
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble
import pandas as pd
import matplotlib.pyplot as plt
import nltk
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize

In [16]:
df_csv = pd.read_csv('datasets/gab_dataset.csv',error_bad_lines=False)
# df_csv = df_csv.dropna()

In [17]:
df_csv.head(10)

,gab_commnet,label
0,i joined gab to remind myself how retarded jew...,1
1,this is what the left is really scared of. htt...,0
2,that literally looks like a monkey. why are we...,0
3,dumb cunt,1
4,it makes you an asshole.,0
5,"give it to a soldier who has defend it , then ...",1
6,"so they manage to provide a whole lot of data,...",0
7,yup. i cant smoke in a bar because it's harmfu...,1
8,"hi there, i,m keith, i hope you are doing well...",0
9,fuck off,0


In [18]:
# # Read data files
comments_attack=pd.DataFrame()
comments_attack['comment']=df_csv['gab_commnet']
comments_attack['label']=df_csv['label']
dataframe = comments_attack

In [19]:
# Text preprocessing
dataframe['comment'] = dataframe['comment'].apply(lambda x: x.replace("NEWLINE_TOKEN", " "))
dataframe['comment'] = dataframe['comment'].apply(lambda x: x.replace("TAB_TOKEN", " "))
dataframe['comment'] = dataframe['comment'].apply(lambda x: x.lower())
# dataframe['comment'] = dataframe['comment'].apply((lambda x: re.sub('[/(){}\[\]\|@,;]','',x)))
# # dataframe['comment'] = dataframe['comment'].apply((lambda x: re.sub('"',' ',x)))
# # dataframe['comment'] = dataframe['comment'].apply((lambda x: re.sub('”',' ',x)))


In [20]:
len(dataframe['label'])

29417

In [21]:
print(dataframe['comment'])

0        i joined gab to remind myself how retarded jew...
1        this is what the left is really scared of. htt...
2        that literally looks like a monkey. why are we...
3                                                dumb cunt
4                                 it makes you an asshole.
                               ...                        
29412                                       fucking retard
29413    twitter banned me and reported me to the fbi f...
29414    i was routinely suspended for saying fuck, and...
29415     they got me for dropping the cunt bomb... a lot.
29416    how absolutely unbelievably fascinating? a pak...
Name: comment, Length: 29417, dtype: object


In [22]:
#split the data into training and validation sets
train_x, valid_x, train_y, valid_y = train_test_split(dataframe['comment'], dataframe['label'], test_size=0.2, random_state=42)

In [23]:
print('train comments length: ',len(train_x))
print('test comments length: ',len(valid_x))

train comments length:  23533
test comments length:  5884


In [24]:
# word level tf-idf
tfidf_vect = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=5000)
tfidf_vect.fit(dataframe['comment'])
xtrain_tfidf =  tfidf_vect.transform(train_x)
xvalid_tfidf =  tfidf_vect.transform(valid_x)

# ngram level tf-idf 
tfidf_vect_ngram = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', ngram_range=(3,4), max_features=5000)
tfidf_vect_ngram.fit(dataframe['comment'])
xtrain_tfidf_ngram =  tfidf_vect_ngram.transform(train_x)
xvalid_tfidf_ngram =  tfidf_vect_ngram.transform(valid_x)

# characters level tf-idf
tfidf_vect_ngram_chars = TfidfVectorizer(analyzer='char', token_pattern=r'\w{1,}', ngram_range=(2,3), max_features=5000)
tfidf_vect_ngram_chars.fit(dataframe['comment'])
xtrain_tfidf_ngram_chars =  tfidf_vect_ngram_chars.transform(train_x) 
xvalid_tfidf_ngram_chars =  tfidf_vect_ngram_chars.transform(valid_x)



C:\Users\mjahan18\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:501: UserWarning: The parameter 'token_pattern' will not be used since 'analyzer' != 'word'
  warnings.warn("The parameter 'token_pattern' will not be used"


In [25]:
def train_model(classifier, xtrain, ytrain, xvalid, yvalid): 
    # fit the training dataset on the classifier
    classifier.fit(xtrain, ytrain)
    # predict the labels on validation dataset
    predictions = classifier.predict(xvalid)     
    accuracy = metrics.accuracy_score(predictions, yvalid)
    f1score = metrics.f1_score(yvalid, predictions, average='weighted')
    return accuracy, f1score

In [26]:
# Naive Bayes on Word Level TF IDF Vectors
accuracy, f1score = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf, train_y, xvalid_tfidf, valid_y)
print("NB, WordLevel TF-IDF:   accuracy: %s     f1 score: %s"% (accuracy,f1score))

# Naive Bayes on Ngram Level TF IDF Vectors
accuracy, f1score = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram, valid_y)
print("NB, N-Gram Vectors:   accuracy: %s     f1 score: %s"% (accuracy,f1score))

# Naive Bayes on Character Level TF IDF Vectors
accuracy, f1score = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf_ngram_chars, train_y, xvalid_tfidf_ngram_chars, valid_y)
print("NB, CharLevel Vectors:   accuracy: %s   f1 score: %s"% (accuracy,f1score))

NB, WordLevel TF-IDF:   accuracy: 0.8122025832766825     f1 score: 0.8123488162537877
NB, N-Gram Vectors:   accuracy: 0.6150577838205302     f1 score: 0.5988413100058025
NB, CharLevel Vectors:   accuracy: 0.7834806254248811   f1 score: 0.7835457672065336


In [27]:
# Linear Classifier on Word Level TF IDF Vectors
accuracy, f1score = train_model(linear_model.LogisticRegression(), xtrain_tfidf, train_y, xvalid_tfidf, valid_y)
print("LR, WordLevel TF-IDF:   accuracy: %s   f1 score: %s"% (accuracy,f1score))

# Linear Classifier on Ngram Level TF IDF Vectors
accuracy, f1score = train_model(linear_model.LogisticRegression(), xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram, valid_y)
print("LR, N-Gram Vectors:   accuracy: %s   f1 score: %s"% (accuracy,f1score))

# Linear Classifier on Character Level TF IDF Vectors
accuracy, f1score = train_model(linear_model.LogisticRegression(), xtrain_tfidf_ngram_chars, train_y, xvalid_tfidf_ngram_chars, valid_y)
print("LR, CharLevel Vectors:   accuracy: %s   f1 score: %s"% (accuracy,f1score))

LR, WordLevel TF-IDF:   accuracy: 0.8966689326988443   f1 score: 0.8960609080183438


C:\Users\mjahan18\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LR, N-Gram Vectors:   accuracy: 0.6096193065941536   f1 score: 0.588390158114091
LR, CharLevel Vectors:   accuracy: 0.8880013596193066   f1 score: 0.8874732395894269
